## Before running

This is the code for deriving the $q$ and $u$ values of the asteroids taken by the Andalucia Faint Object Spectrograph and Camera (ALFOSC) with the FAPOL polarimeter <br>
on the 2.56-m Nordic Optical Telescope at the Observatorio del Roque de los Muchachos, La Palma.<br><br>


1. 
 * Input file:  
    * Phot_{DATE}_{Object_name}.csv         Photometric result of each images generated by 2.FAPOL_aperture_photometry.ipynb.<br> 
 
 * Outout file:
     * Pol_{DATE}_{Object_name}.csv         Polarimetric result of each set<br> 


<br><br><br>    







### Import packages and define the function

In [95]:

import os
import pandas as pd
import numpy as np
from astropy.time import Time
from astroquery.jplhorizons import Horizons



def NOT_qu(kappa,err_kappa):

    k_0 = kappa[0]
    k_22 = kappa[1]
    k_45 = kappa[2]
    k_67 = kappa[3]

    ek_0 = err_kappa[0]
    ek_22 = err_kappa[1]
    ek_45 = err_kappa[2]
    ek_67 = err_kappa[3]

    aQ = np.sqrt(k_0/k_45)
    aU = np.sqrt(k_22/k_67)

    q = (1-aQ)/(1+aQ) #Q/I
    u = (1-aU)/(1+aU) #U/I

    q_ran = (aQ/((aQ + 1)**2)  *  np.sqrt(ek_0 + ek_45))
    u_ran = (aU/((aU + 1)**2)  *  np.sqrt(ek_22 + ek_67))


    ###==================== 
    ## Correct Efficiency
    ###==================== 
    eff = 1
    efferr = 0

    qq = q/eff
    uu = u/eff

    #random error of corrected q,u
    qq_ran = q_ran/eff
    uu_ran = u_ran/eff

    #the systematic errors
    qq_sys = np.abs(q)*efferr/eff
    uu_sys = np.abs(u)*efferr/eff  


    ###==================== 
    ## Correc Instrumental polarization
    ###====================     
    q_inst = -0.00047
    u_inst = -0.00042
    eq_inst =0.0007
    eu_inst = 0.0011
    
    
    qqq = qq - q_inst
    uuu = uu - u_inst

    #random error of corrected q,u
    qqq_ran = qq_ran
    uuu_ran = uu_ran    

    #the systematic errors    
    qqq_sys = np.sqrt( qq_sys**2 + eq_inst**2)
    uuu_sys = np.sqrt( uu_sys**2 + eu_inst**2)    

    ###==================== 
    ## Transform_CelestialCoord
    ###====================    
    the = -92.30 #theta_offset
    the_err = 0.060

    theta = the
    theta = np.deg2rad(theta)
    the_err = np.deg2rad(the_err)


    qqqq = qqq * np.cos(2*theta) + uuu*np.sin(2*theta)
    uuuu = -qqq * np.sin(2*theta) + uuu*np.cos(2*theta)

    qqqq_ran = np.sqrt( (qqq_ran*np.cos(2*theta))**2 + (uuu_ran*np.sin(2*theta))**2 ) 
    uuuu_ran = np.sqrt( (qqq_ran*np.sin(2*theta))**2 + (uuu_ran*np.cos(2*theta))**2 ) 

    qqqq_sys = np.sqrt( (qqq_sys*np.cos(2*theta))**2 + \
                        (uuu_sys*np.sin(2*theta))**2 + \
                        (np.pi/180*2*uuuu*the_err)**2 )
    uuuu_sys = np.sqrt( (qqq_sys*np.sin(2*theta))**2 + \
                        (uuu_sys*np.cos(2*theta))**2 + \
                        (np.pi/180*2*qqqq*the_err)**2 ) 
    return(qqqq, qqqq_ran, qqqq_sys, uuuu, uuuu_ran, uuuu_sys)

def weight(x,err):
    x = np.array(x)
    err = np.array(err)

    w = 1/err**2
    sumW = np.sum(w)
    weight = w/sumW
    
    xav = np.sum(weight*x)
    Err = 1/np.sqrt(sumW)
    
    return(xav,Err)

def extract_exp(LIST):
    hey = []
    for i in LIST:
        hey.append(list(i)[0])
    return(set(hey))    

        


### Input value for deriving $q$ and $u$

In [96]:
path = os.path.join('/home','judy','Downloads/P64_phaethon_polarimetry','211114','target','reduced') #Where your Phot_{DATE}_{Object_name}.csv is saved
filename = os.path.join(path,'Phot_2021_11_15_Phaethon.csv')
file = pd.read_csv(filename)
print(filename)
file

/home/judy/Downloads/P64_phaethon_polarimetry/211114/target/reduced/Phot_2021_11_15_Phaethon.csv


,Unnamed: 0,Filename,Object,DATE,HWPANG,Filter,TIME,JD,EXP [s],Flux_o,...,FWHM_e,Aper_o [pix],Aper_e [pix],Ann,Ann_out,Sky_o,eSky_o,Sky_e,eSky_e,Airmass
0,0,ALEk140230.Phaethon.180.0.fits,Phaethon 180deg,2021-11-15,180.0,R_Bes 650_130,2021-11-15T01:50:19.613,2.459534e+06,180.0,736780.9,...,6.7,8.5,10.4,17.0,47.0,5511.1,190.3,5489.9,185.3,1.03
1,1,ALEk140231.Phaethon.202.5.fits,Phaethon 202.5deg,2021-11-15,202.5,R_Bes 650_130,2021-11-15T01:53:27.186,2.459534e+06,180.0,738277.9,...,7.1,8.7,11.0,18.0,48.0,5469.6,191.6,5460.7,185.4,1.03
2,2,ALEk140232.Phaethon.225.0.fits,Phaethon 225deg,2021-11-15,225.0,R_Bes 650_130,2021-11-15T01:56:34.878,2.459534e+06,180.0,756159.2,...,6.8,8.9,10.5,17.0,47.0,5427.3,188.8,5411.3,186.2,1.03
3,3,ALEk140233.Phaethon.247.5.fits,Phaethon 247.5deg,2021-11-15,247.5,R_Bes 650_130,2021-11-15T01:59:42.365,2.459534e+06,180.0,749096.0,...,7.0,9.4,10.9,18.0,48.0,5480.5,189.8,5468.9,185.3,1.04
4,4,ALEk140238.Phaethon.0.0.fits,Phaethon 0deg,2021-11-15,0.0,R_Bes 650_130,2021-11-15T02:15:21.968,2.459534e+06,180.0,748868.9,...,7.9,11.3,12.2,20.0,50.0,5455.9,185.4,5451.8,184.8,1.05
5,5,ALEk140239.Phaethon.22.5.fits,Phaethon 22.5deg,2021-11-15,22.5,R_Bes 650_130,2021-11-15T02:18:30.300,2.459534e+06,180.0,740752.2,...,8.0,10.5,12.5,20.0,50.0,5368.0,187.3,5376.7,186.5,1.05
6,6,ALEk140240.Phaethon.45.0.fits,Phaethon 45deg,2021-11-15,45.0,R_Bes 650_130,2021-11-15T02:21:38.402,2.459534e+06,180.0,753111.9,...,7.3,9.3,11.3,18.0,48.0,5402.2,182.4,5405.9,180.9,1.06
7,7,ALEk140241.Phaethon.67.5.fits,Phaethon 67.5deg,2021-11-15,67.5,R_Bes 650_130,2021-11-15T02:24:46.576,2.459534e+06,180.0,735216.7,...,7.5,9.7,11.6,19.0,49.0,5338.8,184.2,5333.8,182.3,1.06


### Deriving $q$ and $u$

In [99]:
#======================================#
#             Polarimetry              #
#======================================#    
Phot = file
Pol_log = pd.DataFrame({})
order = np.arange(0,len(Phot),4)        


for i in order:
    Flux_0_e = Phot['Flux_e'].values[i]
    Flux_0_o = Phot['Flux_o'].values[i]
    eFlux_0_e = Phot['eFlux_e'].values[i]
    eFlux_0_o = Phot['eFlux_o'].values[i]
    err_0 = (Flux_0_e/Flux_0_o**2 * eFlux_0_o)**2 + (1/Flux_0_o * eFlux_0_e)**2

    Flux_22_e = Phot['Flux_e'].values[i+1]
    Flux_22_o = Phot['Flux_o'].values[i+1]
    eFlux_22_e = Phot['eFlux_e'].values[i+1]
    eFlux_22_o = Phot['eFlux_o'].values[i+1]
    err_22 = (Flux_22_e/Flux_22_o**2 * eFlux_22_o)**2 + (1/Flux_22_o * eFlux_22_e)**2    

    Flux_45_e = Phot['Flux_e'].values[i+2]
    Flux_45_o = Phot['Flux_o'].values[i+2]
    eFlux_45_e = Phot['eFlux_e'].values[i+2]
    eFlux_45_o = Phot['eFlux_o'].values[i+2]
    err_45 = (Flux_45_e/Flux_45_o**2 * eFlux_45_o)**2 + (1/Flux_45_o * eFlux_45_e)**2

    Flux_67_e = Phot['Flux_e'].values[i+3]
    Flux_67_o = Phot['Flux_o'].values[i+3]
    eFlux_67_e = Phot['eFlux_e'].values[i+3]
    eFlux_67_o = Phot['eFlux_o'].values[i+3]
    err_67 = (Flux_67_e/Flux_67_o**2 * eFlux_67_o)**2 + (1/Flux_67_o * eFlux_67_e)**2

    if Phot['HWPANG'].values[i]%90 !=0:
        print('! HWPANG is not arranged. Check '+Phot['Filename'])
        break
    if Phot['HWPANG'].values[i+1]%90 !=22.5:
        print('! HWPANG is not arranged. Check '+Phot['Filename'])
        break
    if Phot['HWPANG'].values[i+2]%90 !=45:
        print('! HWPANG is not arranged. Check '+Phot['Filename'])
        break
    if Phot['HWPANG'].values[i+3]%90 !=67.5:
        print('! HWPANG is not arranged. Check '+Phot['Filename'])
        break        

    kappa = [Flux_0_e/Flux_0_o, Flux_22_e/Flux_22_o, Flux_45_e/Flux_45_o, Flux_67_e/Flux_67_o]
    ekappa = [err_0, err_22,  err_45, err_67]


    q, ran_q, sys_q, u, ran_u, sys_u = NOT_qu(kappa, ekappa)
    eq = np.sqrt(ran_q**2 + sys_q**2)
    eu = np.sqrt(ran_u**2 + sys_u**2)

    P = np.sqrt(q**2 + u**2)
    P_ran = np.sqrt( (q*ran_q)**2 + (u*ran_u)**2 )/P
    P_sys = np.sqrt( (q*sys_q)**2 + (u*sys_u)**2 )/P
    P_error = np.sqrt(P_ran**2 + P_sys**2) #Polarization error  
    Theta = np.rad2deg(1/2* np.arctan2(u,q))    
    if P**2 > P_ran**2:
        print('Random error bias correction is done.')
        P_cor = np.sqrt(P**2 - P_ran**2)
        ran_PolAng = 1/2 * 180/3.14 * P_ran/P_cor
        sys_PolAng = 1/2 * 180/3.14 * P_sys/P_cor
        eTheta = np.sqrt(ran_PolAng**2 + sys_PolAng**2)

    elif P**2 < P_ran**2 :
        print('Due to P < randome error, random error bias correction is NOT done.')
        P_cor = 0    
        ran_PolAng = 51.96
        sys_PolAng = 51.96
        eTheta = 51.96

    if Theta < 0:
        Theta = Theta + 180
    elif Theta > 180:
        Theta = Theta - 180

    filename = Phot['Filename'].values[i].split('.')[0]+'~'+\
    Phot['Filename'].values[i+3].split('.')[0]+'.'+Phot['Filename'].values[i+3].split('.')[1].split('_')[0]+'.fits'           
    Set_num = int(i/4+1)
    JD = np.mean(Phot['JD'].values[i:i+4])
    UT = Time(JD,format='jd').isot
    UT = UT.split('T')[0] + ' ' + UT.split('T')[1][:5]

    Exp = set(Phot['EXP [s]'].values[i:i+4])
    Airmass =  np.mean(Phot['Airmass'].values[i:i+4])
    Object = Phot['Object'].values[i].split('_')[0]
    Date = Phot['DATE'].values[i]
    SNR = (np.mean(Phot['SNR_o'].values[i:i+4])+ np.mean(Phot['SNR_e'].values[i:i+4]))/2
    HWPANG = Phot['HWPANG'].values[i:i+4]


    FWHM = np.mean([Phot['FWHM_o'].values[i],Phot['FWHM_e'].values[i]])
    Aper = np.mean([Phot['Aper_e [pix]'].values[i],Phot['Aper_o [pix]'].values[i]])
    #Translate to the scattering plane==============================
    obj = Horizons(id=3200,location='Z23', epochs=JD)
    eph = obj.ephemerides()
    psANG = eph['sunTargetPA'][0] #[deg]
    pA = eph['alpha'][0] #[deg]    

    if psANG + 90 < 180:
        pi = psANG + 90
    else:
        pi = psANG - 90

    theta_r = Theta - pi
    Pr = P_cor * np.cos(2*np.deg2rad(theta_r))



    Pol_log = Pol_log.append({'Filename':filename,
                                'Set':Set_num,
                                'JD':JD,
                              'UT':UT,
                              'alpha':pA,
                              'SunTargetPA':psANG,
                                'EXP [s]':Exp,
                                'Airmass':Airmass,
                              'HWPANG':HWPANG,
                                'Object':Object,
                              'Filter':'R_Bes 650_130',
                                'Date':Date,
                                'SNR':round(SNR,1),
                                'Pr [%]':Pr*100,
                                'Theta_r':theta_r,
                                'P [%]':P_cor*100,
                                'sys_P [%]':P_sys*100,
                                'ran_P [%]':P_ran*100,
                                'eP [%]':P_error*100,
                                'q':q,
                                'u':u,
                                'ran_q':ran_q,
                                'sys_q':sys_q,
                                'ran_u':ran_u,
                                'sys_u':sys_u,
                                'eq':eq,
                                'eu':eu,
                                'Theta':Theta,
                                'eTheta':eTheta,
                                'sys_Theta':sys_PolAng,
                                'ran_Theta':ran_PolAng,
                              'FWHM':FWHM,
                             'Aper [pix]':Aper},
                             ignore_index=True)    


q_av, ranq_av = weight(Pol_log['q'],Pol_log['ran_q'])
u_av, ranu_av = weight(Pol_log['u'],Pol_log['ran_u'])
sysq_av = np.mean(Pol_log['sys_q'])
sysu_av = np.mean(Pol_log['sys_u'])
errq_av = (ranq_av**2 + sysq_av**2)**0.5
erru_av = (ranu_av**2 + sysu_av**2)**0.5

P = np.sqrt(q_av**2+u_av**2)
ran_P = np.sqrt((q_av*ranq_av)**2 + (u_av*ranu_av)**2)/P
sys_P = np.sqrt((q_av*sysq_av)**2 + (u_av*sysu_av)**2)/P
eP = np.sqrt(ran_P**2 + sys_P**2)

Theta = 1/2*np.rad2deg(np.arctan2(u_av,q_av))
if P**2 - ran_P**2 < 0:
    Pcor = 0
    ran_PolAng = 51.96
    sys_PolAng = 1/2 * 180/3.14 * P_sys/Pcor
    PolAng_error = 51.96
    # Naghizadeh-Khouei & Clarke 1993
else:
    Pcor = np.sqrt(P**2 - ran_P**2)
    ran_PolAng = 1/2 * 180/3.14 * P_ran/Pcor
    sys_PolAng = 1/2 * 180/3.14 * P_sys/Pcor
    PolAng_error = np.sqrt(ran_PolAng**2 + sys_PolAng**2)      


if Theta < 0:
    Theta = Theta + 180
elif Theta > 180:
    Theta = Theta - 180    



JD = np.mean(Pol_log['JD'])
Exp = extract_exp(Pol_log['EXP [s]'].values)
Airmass = '{0:.1f}--{1:.1f}'.format(min(Pol_log['Airmass'].values),max(Pol_log['Airmass'].values))
Object = Pol_log['Object'].values[0]
Date = Pol_log['Date'].values[0]
SNR = '{0:.1f}--{1:.1f}'.format(min(Pol_log['SNR'].values),max(Pol_log['SNR'].values))

FWHM = np.mean(Pol_log['FWHM'])
Aper = np.mean(Pol_log['Aper [pix]'])

UT = Pol_log['UT'].values[0] +'-'+Pol_log['UT'].values[-1].split(' ')[-1]


#Translate to the scattering plane==============================
obj = Horizons(id=3200,location='Z23', epochs=JD)
eph = obj.ephemerides()
psANG = eph['sunTargetPA'][0] #[deg]
pA = eph['alpha'][0] #[deg]    

if psANG + 90 < 180:
    pi = psANG + 90
else:
    pi = psANG - 90

theta_r = Theta - pi
Pr = Pcor * np.cos(2*np.deg2rad(theta_r))
Pol_log = Pol_log.append({'Filename':'Weighted Mean',
                          'Set':'Total {0}'.format(Set_num),
                            'JD':JD,
                          'UT':UT,
                            'EXP [s]':Exp,
                            'Airmass':Airmass,
                          'Filter':'R_Bes 650_130',
                            'Object':Object,
                            'Date':Date,
                            'SNR':SNR,
                            'Pr [%]':Pr*100,
                            'Theta_r':theta_r,
                            'P [%]':Pcor*100,
                            'sys_P [%]':sys_P*100,
                            'ran_P [%]':ran_P*100,
                            'eP [%]':eP*100,
                            'q':q_av,
                            'u':u_av,
                            'ran_q':ranq_av,
                            'sys_q':sysq_av,
                            'ran_u':ranu_av,
                            'sys_u':sysu_av,
                            'eq':eq,
                            'eu':eu,
                            'Theta':Theta,
                            'eTheta':PolAng_error,
                            'sys_Theta':sys_PolAng,
                            'ran_Theta':ran_PolAng,
                              'alpha':pA,
                              'SunTargetPA':psANG,
                         'FWHM':FWHM,
                          'Aper [pix]':Aper},
                            ignore_index=True)       

new_index = ['Filename','Date', 'UT','Set','HWPANG','Object',
             'alpha','Pr [%]','eP [%]','P [%]','sys_P [%]','ran_P [%]',
             'q','u','ran_q','sys_q','ran_u',
             'sys_u','eq','eu','Theta_r','Theta',
             'eTheta','sys_Theta','ran_Theta','EXP [s]',
             'SNR','Filter','JD','Airmass','SunTargetPA','FWHM',
            'Aper [pix]']
Pol_log = Pol_log.reindex(columns = new_index)  
Pol_log = Pol_log.round({'alpha':2,'P [%]':2,'sys_P [%]':2,'ran_P [%]':2,'eP [%]':2,
                            'q':4,'u':4,'ran_q':4,'sys_q':4,'ran_u':4,
                             'sys_u':4,'eq':4,'eu':4,'Theta':3, 'eTheta':3,
                             'sys_Theta':3,'ran_Theta':3,'SNR':1,'SunTargetPA':2,
                        'FWHM':1,'Aper_o [pix]':1, 'Aper_e [pix]':1})

Object_name = Object.split(' ')[0]
DATE = Date.replace('-','_')
FILENAME = 'Pol_'+DATE+'_'+Object_name+'.csv'
FILENAME = os.path.join(path,FILENAME)

Pol_log.to_csv(FILENAME)        

Random error bias correction is done.
Random error bias correction is done.


In [100]:
Pol_log

,Filename,Date,UT,Set,HWPANG,Object,alpha,Pr [%],eP [%],P [%],...,sys_Theta,ran_Theta,EXP [s],SNR,Filter,JD,Airmass,SunTargetPA,FWHM,Aper [pix]
0,ALEk140230~ALEk140233.Phaethon.fits,2021-11-15,2021-11-15 01:55,1,"[180.0, 202.5, 225.0, 247.5]",Phaethon 180deg,8.95,-1.458338,0.24,1.46,...,1.427,4.464,{180.0},227.6,R_Bes 650_130,2.459534e+06,1.0325,186.44,6.1,9.45
1,ALEk140238~ALEk140241.Phaethon.fits,2021-11-15,2021-11-15 02:20,2,"[0.0, 22.5, 45.0, 67.5]",Phaethon 0deg,8.94,-1.297696,0.27,1.51,...,1.424,4.835,{180.0},208.1,R_Bes 650_130,2.459534e+06,1.055,186.36,7.6,11.75
2,Weighted Mean,2021-11-15,2021-11-15 01:55-02:20,Total 2,NaN,Phaethon 180deg,8.95,-1.393982,0.18,1.44,...,1.495,5.076,{180.0},208.1--227.6,R_Bes 650_130,2.459534e+06,1.0--1.1,186.40,6.8,10.60
